# Web Scraping from Indeed.com & Predicting Salaries

This project includes collecting data by scraping Indeed.com and then building a binary classifier.

We collect salary information on data science jobs in a variety of markets. Then using the location, title, and summary of the job we use listings with salary information to build a model and predict additional salaries.

## Scraping job listings from Indeed.com

In [2]:
# Import packages
import requests
import bs4
from bs4 import BeautifulSoup
import re    # string manipulations
import pandas as pd
import numpy as np
import datetime

### Functions to extract location, company, job title, and salary

In [ ]:
def extract_title_from_result(result):
# find the title, if get error, return NONE
    try:
        result_txt = result.find('a',attrs={'data-tn-element':'jobTitle'}).text
    except:
        result_txt='NONE'
    return result_txt

In [ ]:
def extract_company_from_result(result):
# find the company, if get error, return NONE
    try:
        result_txt = result.find('span',class_='company').text
    except:
        result_txt='NONE'
    result_txt= re.sub('\n', '', result_txt).strip()   # strip '\n' and leading blanks
    return result_txt

In [ ]:
def extract_location_from_result(result):
# find the location, if get error, return NONE
    try:
        result_txt = result.find('span',class_='location').text
    except:
        location_txt='NONE'
    return result_txt

In [ ]:
def extract_salary_from_result(result):
# find the salary, if get error, return NONE
    try:
        result_txt = result.find('span',attrs={'class':'no-wrap'}).text
    except:
        result_txt='NONE'
    return result_txt

In [ ]:
def extract_description_from_result(result):
# find the description, if get error, return NONE
    try:
        result_txt = result.find('span',attrs={'class':'summary'}).text
        result_txt= re.sub('\n', '', result_txt).strip()   # strip '\n' and leading blanks
    except:
        result_txt='NONE'
    return result_txt

### Scrape Indeed Data Science jobs in multiple cities

In [ ]:
# Base URL to scrape from
url_template = "http://www.indeed.com/jobs?q=data+scientist+%2420%2C000&l={}&start={}"

max_results_per_city = 2000

# Initialize lists
titles=[]
companies=[]
locations=[]
salaries=[]
descriptions=[]

# Loop to perform scraping by city
for city in set(['New+York', 'Chicago', 'San+Francisco', 'Austin', 'Seattle', 
    'Los+Angeles', 'Philadelphia', 'Atlanta', 'Dallas', 'Pittsburgh', 
    'Portland', 'Phoenix', 'Denver', 'Houston', 'Miami', 'Raleigh']):

    print city   # so can see status
# Loop to perform scraping by page within each city
    for start in range(0, max_results_per_city, 10):
                
        print start # so can see status
        print('Timestamp: {:%Y-%m-%d %H:%M:%S}'.format(datetime.datetime.now()))
       
        url= url_template.format(city,start)   # Instantiate URL
        html=requests.get(url)                 # get HTML
        b=BeautifulSoup(html.text)             # pass through Beautiful Soup
        
# Work through the job listings one at a time
        results = b.find_all('div',attrs={'class':' row result'})

# Extract the individual fields from each result using the functions written
        for r in results:
            titles.append(extract_title_from_result(r))
            companies.append(extract_company_from_result(r))
            locations.append(extract_location_from_result(r))
            salaries.append(extract_salary_from_result(r))
            descriptions.append(extract_description_from_result(r))   # in case do Bonus

# Once have all the data, zip the fields together into 'listings'
listings_list = zip(titles, companies,locations,salaries,descriptions)

# Put the listings into a data frame
column_names = ['Title','Company','Location','Salary','Description']   # Column names
listings= pd.DataFrame(listings_list,columns=column_names)          # Put into dataframe

In [ ]:
# Write to csv file so don't lose
file_name='Data Science Jobs 2000.csv'
listings.to_csv(file_name,encoding='utf-8',index=False)
    

### Clean up the data

1. Only a small number of the scraped results have salary information - only these will be used for modeling.
1. Some of the salaries are not yearly but hourly or weekly, remove them
1. Some of the entries may be duplicated
1. The salaries are given as text and usually with ranges.

In [ ]:
# Create a new data frame with rows where salaries contain the phrase 'a year'
# This removes all the rows with salary = None or per month or per hour salaries
listings2 = listings[listings.Salary.str.contains('a year')]

# Remove duplicates
listings2.drop_duplicates(inplace=True)

In [ ]:
# Function convert_salary removes non-numerics, trims off spaces, and convert strings to integers
def convert_salary(salary_str):
    salary_str = re.sub('[^0-9]','',salary_str).strip()       # strip '$' and ',' and remove spaces
    return int(salary_str)

In [ ]:
# Function clean_salary takes in salary string, removes superfluous characters,
# averages a range (if required), and converts to integer
def clean_salary(salary_txt):

# Remove the words ' a year'
    salary_txt= re.sub('a year', '', salary_txt).strip()   # strip 'a year' and trailing blanks

# Check if contains a range (has a '-').  If so, divide in two parts and take average
    if '-' in salary_txt:
        position = salary_txt.find('-')
        salary1 = salary_txt[0:position-1]   # before the '-'
        salary2 = salary_txt[position+1:]    # after the '-'
        salary = (convert_salary(salary1) + convert_salary(salary2))/2   #Convert each to integer, take average       
# No salary range so just clean up the salary field
    else:
        salary = (convert_salary(salary_txt))  # clean up and convert to integer
   
    return salary

In [ ]:
# Run clean_salary on salary column in data frame to get single integer value
listings2['Salary']=listings2['Salary'].apply(clean_salary)

#### Save results as a CSV

In [ ]:
# Export to csv
file_name='Data Science Jobs 2000clean.csv'
listings2.to_csv(file_name,encoding='utf-8',index=False)


## Predict salaries using Random Forests and Logistic Regression

#### Load in the the data of scraped salaries

In [ ]:
# Read in the dataframe of cleaned up listings written to csv above
listings2=pd.read_csv(file_name,header=0,index_col=0)

# Reset the index values
listings2.reset_index(inplace=True)

#### Predicting a binary variable - whether the salary was low or high. 
Compute the median salary and create a new binary variable that is true when the salary is high (above the median)

In [ ]:
# compute the median of the salary column
median_sal= np.median(listings2.Salary)
print 'Median Salary is '+str(median_sal)

# Add column to the dataframe that equals 1 if salary is above average and 0 otherwise

# Function that determines if salary is above average for the row
def above_avg(row):
    if row.Salary > median_sal:
        above = 1
    else:
        above = 0
    return above

# Apply the above_avg function to the listings2 dataframe
listings2['High_salary']= listings2.apply(above_avg,axis=1)

###### Add columns to the dataframe for City, State, and Metro area since the location field sometimes contains extraneous information like zip code 

In [ ]:
# City is the parsed out city name.   State is the parsed out state name.
# Metro area is the City field that I manually inspect and modify where needed in the csv file,
# for example, changing Brooklyn to New York.

# Functions that parse the city and state from the location
def parse_city(row):
    position = row.Location.find(',')
    city = row.Location[0:position]   # before the ','
    return city

def parse_state(row):
    position = row.Location.find(',')
    state = row.Location[position+2:position+4]   # after the ',' and space
    return state

# Create City column
listings2['City']= listings2.apply(parse_city,axis=1)

# Create a Metro column based on City.   I'll manually change this in the CSV to the 
# surrounding metro area.  
listings2['Metro']= listings2.apply(parse_city,axis=1)

# Create State column
listings2['State']= listings2.apply(parse_state,axis=1)

#### Create a additional variables for interesting features of a job title.

In [ ]:
# Add columns to the database for words 'Senior', 'Manager', 'Analyst', 'Engineer', and phrase 'Data Scientist'

# Function that finds string in job title
def parse_title(row,string):
    if string in row.Title:
        return 1
    else:
        return 0

# Add column for whether 'Senior' is in title
word = 'Senior'
listings2['Senior']= listings2.apply(parse_title,args=(word,),axis=1)

# Add column for whether 'Manager' is in title
word = 'Manager'
listings2['Manager']= listings2.apply(parse_title,args=(word,),axis=1)

# Add column for whether 'Data Scientist' is in title
word = 'Data Scientist'
listings2['Data_Scientist']= listings2.apply(parse_title,args=(word,),axis=1)

# Add column for whether 'Assistant' is in title
word = 'Assistant'
listings2['Assistant']= listings2.apply(parse_title,args=(word,),axis=1)

# Add column for whether 'Analyst' is in title
word = 'Analyst'
listings2['Analyst']= listings2.apply(parse_title,args=(word,),axis=1)

# Add column for whether 'Engineer' is in title
word = 'Engineer'
listings2['Engineer']= listings2.apply(parse_title,args=(word,),axis=1)


In [ ]:
# Export results to csv
file_name='Data Science Jobs 2000addcol.csv'
listings2.to_csv(file_name,index=False)

#### Create a Random Forest model to predict High/Low salary using Sklearn. Start by ONLY using the location as a feature. 

In [5]:
# Read in the dataframe of cleaned up listings written to csv above
file_name='Data Science Jobs 2000addcol.csv'
listings2=pd.read_csv(file_name,header=0)

In [6]:
# Create random forest model
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

rfc = RandomForestClassifier(verbose=1)       #initialize

In [7]:
# Build random forests to predict whether high salary based on Metro
X = pd.get_dummies(listings2.Metro)   # encode binary variables for Metro
y=listings2.High_salary               # y is a series of integers

model_rf= rfc.fit(X,y)                # fit model

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished


In [8]:
# Do cross validation and summarize the scores
from sklearn.model_selection import cross_val_score
rfscores=cross_val_score(model_rf,X,y,cv=5)
print rfscores
print "{} Score:\t{:0.3} ± {:0.3}".format("Random Forest", rfscores.mean().round(3), rfscores.std().round(3))

[ 0.25423729  0.49152542  0.52542373  0.48275862  0.56140351]
Random Forest Score:	0.463 ± 0.108


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished


Bias isn't much above 50%. (NOTE - varies a lot when rerun.)  Variance is fair.  
Metro area by itself is not a good model

In [9]:
# Build a new Random Forest with additional features. 
Xrf2 = pd.get_dummies(listings2.drop(['Title','Company','Location','Salary','Description','High_salary','City','State'], axis=1))
y=listings2.High_salary              # y is same as above - a series of integers

model_rf2= rfc.fit(Xrf2,y)

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished


In [10]:
# Do cross validation and summarize the scores
rf2scores=cross_val_score(model_rf2,Xrf2,y,cv=5)
print rf2scores
print "{} Score:\t{:0.3} ± {:0.3}".format("Random Forest", rf2scores.mean().round(3), rf2scores.std().round(3))


[ 0.77966102  0.79661017  0.81355932  0.72413793  0.71929825]
Random Forest Score:	0.767 ± 0.038


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished


Vs. just using location (Metro), accuracy has gone up from 58% to 74%, a big improvement
Variance has gone down also     #NOTE - actual results changing run to run
This looks like a reasonable model, but would like better

In [11]:
# Look at the resulting feature importance to see which have the most impact
feature_importances = pd.DataFrame(rfc.feature_importances_,
                                   index = Xrf2.columns,
                                    columns=['importance']).sort_values('importance',
                                                                        ascending=False)
feature_importances.head(50)


,importance
Data_Scientist,0.313803
Engineer,0.107850
Analyst,0.085949
Metro_San Francisco,0.055675
Metro_Philadelphia,0.054414
Metro_Chicago,0.051483
Senior,0.049022
Metro_Atlanta,0.040860
Metro_New York,0.037115
Metro_Pittsburgh,0.029333


This just indicates which words have the most influence, not if it is + or - influence
I'll determine later which appear to have + vs. - influence using logisitic regression

In [12]:
listings2.columns

Index([u'Title', u'Company', u'Location', u'Salary', u'Description',
       u'High_salary', u'City', u'Metro', u'State', u'Senior', u'Manager',
       u'Data_Scientist', u'Assistant', u'Analyst', u'Engineer'],
      dtype='object')

In [13]:
# Use count-vectorizer to create additional features based on the words in the job titles.

from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer(analyzer="word",stop_words='english',decode_error='ignore')
title_features = cv.fit_transform(listings2.Title)
cv.get_feature_names()

[u'14564',
 u'17071701',
 u'287881',
 u'500',
 u'academic',
 u'access',
 u'account',
 u'accountant',
 u'accounting',
 u'admin',
 u'administration',
 u'administrative',
 u'advanced',
 u'advancement',
 u'advisor',
 u'ai',
 u'aids',
 u'air',
 u'algorithm',
 u'alumni',
 u'anal',
 u'analysis',
 u'analyst',
 u'analytical',
 u'analytics',
 u'animal',
 u'ar',
 u'architect',
 u'asc',
 u'assessment',
 u'assistant',
 u'assoc',
 u'associate',
 u'asst',
 u'assurance',
 u'audit',
 u'aws',
 u'azure',
 u'backend',
 u'banking',
 u'bat',
 u'big',
 u'billion',
 u'bio',
 u'bioinformatics',
 u'biologics',
 u'biology',
 u'biosolids',
 u'biostatistics',
 u'buildout',
 u'bureau',
 u'business',
 u'capital',
 u'cards',
 u'caretaker',
 u'cata',
 u'cellular',
 u'center',
 u'certifying',
 u'chain',
 u'chair',
 u'chemist',
 u'chemistry',
 u'chief',
 u'child',
 u'children',
 u'civil',
 u'client',
 u'clinical',
 u'cloning',
 u'code',
 u'coding',
 u'commiss',
 u'commissioner',
 u'company',
 u'compliance',
 u'computati

Resulting feature names look fairly clean except for few numerics

In [14]:
# Build a new random forest model with location and these new features included.

In [15]:
# Move the featues to an array and a dataframe
title_features_array = title_features.toarray()
title_features_df = pd.DataFrame(title_features_array,columns=cv.get_feature_names())
title_features_df.shape

(292, 349)

In [16]:
listings2['Metro'].shape

(292,)

In [17]:
# For the x variable, concatenate Metro (location) with 
# the new features from the count vectorizer
Xrf3 = pd.concat([pd.get_dummies(listings2.Metro),title_features_df],axis=1)
print Xrf3.shape
Xrf3.columns

(292, 365)


Index([     u'Atlanta',       u'Austin',      u'Chicago',       u'Dallas',
             u'Denver',      u'Houston',  u'Los Angeles',        u'Miami',
           u'New York', u'Philadelphia',
       ...
           u'visionin',         u'vivo',           u'vp',         u'warm',
         u'wastewater',          u'web',      u'welfare',        u'world',
            u'writing',        u'youth'],
      dtype='object', length=365)

In [18]:
# Fit the 3rd random forest model to predict High Salary using Metro (location) and all the words in the titles
y=listings2.High_salary              # y is same as above - a series of integers
model_rf3= rfc.fit(Xrf3,y)

[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished


In [19]:
# Do cross validation and summarize the scores
from sklearn.model_selection import cross_val_score
rf3scores=cross_val_score(model_rf3,Xrf3,y,cv=5)
print rf3scores
print "{} Score:\t{:0.3} ± {:0.3}".format("Random Forest", rf3scores.mean().round(3), rf3scores.std().round(3))


[ 0.77966102  0.76271186  0.83050847  0.81034483  0.73684211]
Random Forest Score:	0.784 ± 0.033


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished


Prediction accuracy improved some with addition of all words in the title, but variance went up slightly 

To summarize (NOTE - values vary slightly from run to run):
With just Metro (Location): Random Forest Score:	0.585 ± 0.072
With the addition of words I chose: Random Forest Score:	0.739 ± 0.037
With the addition of all the words in the title: Random Forest Score:	0.804 ± 0.044 

In [20]:
# Look at which words in the title and which locations had the greatest feature importance
feature_importances = pd.DataFrame(rfc.feature_importances_,
                                   index = Xrf3.columns,
                                    columns=['importance']).sort_values('importance',
                                                                        ascending=False)
feature_importances.head(50)
# This just indicates which words have the most influence, not if it is + or - influence

,importance
data,0.071868
scientist,0.063806
research,0.038561
analyst,0.037576
learning,0.030206
machine,0.028130
Philadelphia,0.023879
New York,0.020670
lead,0.017583
Atlanta,0.015781


In [21]:
# For reference, print the location importance
print 'Location Importances (highest to lowest - varies from run to run)'
print 'Philadelphia: '+str(feature_importances.get_value('Philadelphia','importance'))
print 'New York: '+str(feature_importances.get_value('New York','importance'))
print 'San Francisco: '+str(feature_importances.get_value('San Francisco','importance'))
print 'Atlanta: '+str(feature_importances.get_value('Atlanta','importance'))
print 'Chicago: '+str(feature_importances.get_value('Chicago','importance'))
print 'Pittsburgh: '+str(feature_importances.get_value('Pittsburgh','importance'))
print 'Miami: '+str(feature_importances.get_value('Miami','importance'))
print 'Phoenix: '+str(feature_importances.get_value('Phoenix','importance'))
print 'Austin: '+str(feature_importances.get_value('Austin','importance'))
print 'Portland: '+str(feature_importances.get_value('Portland','importance'))
print 'Houston: '+str(feature_importances.get_value('Houston','importance'))
print 'Denver: '+str(feature_importances.get_value('Denver','importance'))
print 'Seattle: '+str(feature_importances.get_value('Seattle','importance'))
print 'Los Angeles: '+str(feature_importances.get_value('Los Angeles','importance'))
print 'Dallas: '+str(feature_importances.get_value('Dallas','importance'))
print 'Raleigh: '+str(feature_importances.get_value('Raleigh','importance'))

Location Importances (highest to lowest - varies from run to run)
Philadelphia: 0.0238793433402
New York: 0.020670307324
San Francisco: 0.009887976573
Atlanta: 0.015781381455
Chicago: 0.0145235067191
Pittsburgh: 0.0110023142789
Miami: 0.00452748235648
Phoenix: 0.00589379783303
Austin: 0.00609678880759
Portland: 0.0109188165585
Houston: 0.0154586974128
Denver: 0.00197320731529
Seattle: 0.00385032467514
Los Angeles: 0.00423201595799
Dallas: 0.00180150226465
Raleigh: 0.00665545742795


In [22]:
# Ones at the low end of the ranking have little impact so assumably are near the median

#### Repeat the model-building process using Logistic Regression

In [23]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression

In [24]:
# Do logistic regression for Metro (location) only.   Reuse X and y from above
lr = LogisticRegression()       #initialize
model_lr= lr.fit(X,y)

In [25]:
# Do cross validation and summarize the scores
lrscores=cross_val_score(model_lr,X,y,cv=5)
print lrscores
print "{} Score:\t{:0.3} ± {:0.3}".format("Logistic Regression", lrscores.mean().round(3), lrscores.std().round(3))

[ 0.57627119  0.49152542  0.52542373  0.48275862  0.50877193]
Logistic Regression Score:	0.517 ± 0.033


This model with just the location has poor prediction accuracy 
Compare to random forest with same inputs: Random Forest Score:	0.585 ± 0.072
Random forest had better prediction accuracy but higher variance

In [26]:
# Put coefficients into readable format
coeff_vals = pd.DataFrame(lr.coef_.transpose(),index = X.columns,
                                   columns=['weights']).sort_values('weights',
                                                                       ascending=False)
coeff_vals

,weights
San Francisco,1.488790
Philadelphia,1.265302
Chicago,0.984287
Austin,0.922431
Raleigh,0.583647
Seattle,0.451859
Los Angeles,0.239937
New York,-0.118353
Atlanta,-0.166023
Denver,-0.222972


Jobs in San Francisco, Philadelphia, and Chicago pay more than jobs in Pittsburgh, Portland,
Phoenix, and Miami as examples.   Others are in between

In [27]:
# Do logistic regression including the title keywords that I identified
# Reuse Xrf2 and y from above
lr2 = LogisticRegression()       #initialize
model_lr2= lr2.fit(Xrf2,y)

# Do cross validation and summarize the scores
lr2scores=cross_val_score(model_lr2,Xrf2,y,cv=5)
print lr2scores
print "{} Score:\t{:0.3} ± {:0.3}".format("Logistic Regression", lr2scores.mean().round(3), lr2scores.std().round(3))

[ 0.81355932  0.79661017  0.79661017  0.77586207  0.71929825]
Logistic Regression Score:	0.78 ± 0.033


For linear regression, prediction accuracy has increased with similar variance
Comparing to the random forest model with the same inputs: Random Forest Score:	0.739 ± 0.037
This is slightly more accurate than the random forest model with similar variance

In [28]:
# Put coefficients into readable format
coeff_vals2 = pd.DataFrame(lr2.coef_.transpose(),index = Xrf2.columns,
                                   columns=['weights']).sort_values('weights',
                                                                       ascending=False)

# Raise e to the coefficient column to get the likelihood of being above the median
coeff_vals2['Likelihood_Change'] = np.exp(coeff_vals2['weights'])

coeff_vals2

,weights,Likelihood_Change
Data_Scientist,2.420162,11.247680
Metro_San Francisco,1.332194,3.789349
Engineer,1.202103,3.327107
Metro_Philadelphia,1.029181,2.798773
Metro_Austin,0.586749,1.798133
Metro_Seattle,0.505119,1.657183
Senior,0.467914,1.596660
Metro_Chicago,0.357732,1.430083
Manager,0.278444,1.321073
Metro_Los Angeles,0.145275,1.156358


Comparing to the random forest, many of the words that it listed as most impactful are 
showing up with high postive or high negative

In [29]:
# Do logistic regression including all the title words identified by the count vectorizer
# Reuse Xrf3 and y from above
lr3 = LogisticRegression()       #initialize
model_lr3= lr3.fit(Xrf3,y)

# Do cross validation and summarize the scores
lr3scores=cross_val_score(model_lr3,Xrf3,y,cv=5)
print lr3scores
print "{} Score:\t{:0.3} ± {:0.3}".format("Logisitic Regression", lr3scores.mean().round(3), lr3scores.std().round(3))


[ 0.79661017  0.76271186  0.81355932  0.81034483  0.73684211]
Logisitic Regression Score:	0.784 ± 0.03


For logistic regression, this is practically equivalent in accuracy and variance
to the model with the title keywords I identified.   
The random forest model performed slightly more accuarately, with higher variance
Random Forest Score:	0.804 ± 0.044 
(Note about variance - a rerun of the random forest gave score of 0.784 ± 0.014)
So although random forest might be a bit better we can't say so with any certainty

In [30]:
# Put coefficients into readable format
coeff_vals3 = pd.DataFrame(lr3.coef_.transpose(),index = Xrf3.columns,
                                   columns=['weights']).sort_values('weights',
                                                                       ascending=False)
# Raise e to the coefficient column to get the likelihood of being above the median
coeff_vals3['Likelihood_Change'] = np.exp(coeff_vals3['weights'])
coeff_vals3

,weights,Likelihood_Change
quantitative,1.477360,4.381365
data,1.185176,3.271264
director,1.166122,3.209521
machine,1.084979,2.959378
learning,1.084979,2.959378
Philadelphia,0.986667,2.682278
principal,0.981386,2.668150
lead,0.909156,2.482226
scientist,0.868119,2.382425
informatics,0.852438,2.345358


In [31]:
# Much of this lines up with words that the random forest showed had the most influence

##### With Grid search rerun logistic regression with L1 regularization and range of C (inverse of alpha) to try to determine key variables in Title


In [32]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LassoCV

In [33]:
#initialize model, setup parameters
lrt2 = LogisticRegression(penalty='l1')      # Lasso = l1
model_lrt2= lrt2.fit(Xrf3,y)

Cs=np.array([0.001,0.01,0.1,1])

# fit model
gridcv_lrt2=GridSearchCV(estimator=lrt2, param_grid=dict(C=Cs))
gridcv_lrt2.fit(Xrf3,y)

# grid.cv_results_   
gridcv_lrt2.best_estimator_

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [34]:
# Adjust c range and rerun

Cs=np.array([0.5,0.8,1,5,10])

# fit model
gridcv_lrt2=GridSearchCV(estimator=lrt2, param_grid=dict(C=Cs))
gridcv_lrt2.fit(Xrf3,y)

# grid.cv_results_   
gridcv_lrt2.best_estimator_

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [35]:
# Adjust c range and rerun

Cs=np.array([0.9,1,2,3])

# fit model
gridcv_lrt2=GridSearchCV(estimator=lrt2, param_grid=dict(C=Cs))
gridcv_lrt2.fit(Xrf3,y)

# grid.cv_results_   
gridcv_lrt2.best_estimator_

LogisticRegression(C=3.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [36]:
# Adjust c range and rerun

Cs=np.array([2,3,4])

# fit model
gridcv_lrt2=GridSearchCV(estimator=lrt2, param_grid=dict(C=Cs))
gridcv_lrt2.fit(Xrf3,y)

# grid.cv_results_   
gridcv_lrt2.best_estimator_

LogisticRegression(C=3, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [37]:
# look at the results with C = 3

In [38]:
# Want to see associated coefficients into readable format

coeff_valslrt2 = pd.DataFrame(lrt2.coef_.transpose(),index = Xrf3.columns,
                                   columns=['weights']).sort_values('weights',
                                                                       ascending=False)
# Raise e to the coefficient column to get the likelihood of being above the median
coeff_valslrt2['Likelihood_Change'] = np.exp(coeff_valslrt2['weights'])
coeff_valslrt2

,weights,Likelihood_Change
quantitative,2.622498,13.770078
machine,2.010368,7.466062
director,1.751059,5.760698
principal,1.523325,4.587455
data,1.426325,4.163371
lead,1.360396,3.897735
architect,1.078796,2.941136
Philadelphia,1.052868,2.865858
sales,0.854563,2.350346
sr,0.747354,2.111406


In [39]:
# Look at accuracy of the above model
# Do cross validation and summarize the scores
lrt2scores=cross_val_score(model_lrt2,Xrf3,y,cv=5)
print lrt2scores
print "{} Score:\t{:0.3} ± {:0.3}".format("Linear Regression", lrt2scores.mean().round(3), lrt2scores.std().round(3))

[ 0.72881356  0.77966102  0.79661017  0.82758621  0.75438596]
Linear Regression Score:	0.777 ± 0.034


In [40]:
# Similar accuracy to other models. 
# Will use words with non-zero coefficients after lasso regularization as good predictor words
# Write words to file so can visualize in Tableau
coeff_valslrt2.to_csv('final_title_keywords.csv')

In [42]:
# Edited remaining keywords in csv file to those that made sense.   
# Read edited file back in with 23 most impactful keywords from the title

keywordsTitle=pd.read_csv('Title keywords to import.csv',header=0)
keywordsTitle

,Term
0,quantitative
1,machine
2,learning
3,director
4,principal
5,data
6,scientist
7,lead
8,architect
9,sales


In [43]:
# Rerun logistic regression with just these title keywords (no location)

XTitlekey = Xrf3[keywordsTitle['Term'].tolist()]    #X6 is reduced list of keywords
lrTK = LogisticRegression()           
model_lrTK= lrTK.fit(XTitlekey,y)

# Do cross validation and summarize the scores
lrTKscores=cross_val_score(model_lrTK,XTitlekey,y,cv=5)
print lrTKscores
print "{} Score:\t{:0.3} ± {:0.3}".format("Linear Regression", lrTKscores.mean().round(3), lrTKscores.std().round(3))

[ 0.86440678  0.74576271  0.86440678  0.82758621  0.73684211]
Linear Regression Score:	0.808 ± 0.056


80% accuracy, with some variance - about comparable to what have been seeing

In [44]:
# Add information on weights and likelihood
coeff_valslrTK = pd.DataFrame(lrTK.coef_.transpose(),index = XTitlekey.columns,
                                   columns=['weights']).sort_values('weights',
                                                                       ascending=False)
# Raise e to the coefficient column to get the likelihood of being above the median
coeff_valslrTK['Likelihood_Change'] = np.exp(coeff_valslrTK['weights'])
coeff_valslrTK

,weights,Likelihood_Change
quantitative,1.936027,6.931155
director,1.412827,4.107552
data,1.302226,3.677475
machine,1.291843,3.639487
learning,1.291843,3.639487
principal,1.249448,3.488416
lead,1.128660,3.091511
sales,1.028853,2.797855
architect,1.014450,2.757847
scientist,0.843835,2.325268


In [45]:
# Write words to file so can visualize in Tableau
coeff_valslrTK.to_csv('final_title_keywords_no_city.csv')

#### Repeat above for words in the job descriptions. 

In [46]:
# Count vectorize the descriptions
cvd=CountVectorizer(analyzer="word",stop_words='english',decode_error='ignore')
description_features = cvd.fit_transform(listings2.Description)

In [47]:
# Set up the dependent variable input (X4) for the linear regression
# Concatenate Metro (location) with the new features from the count vectorizer
description_features_array = description_features.toarray()
description_features_df = pd.DataFrame(description_features_array,columns=cvd.get_feature_names())

X4 = pd.concat([pd.get_dummies(listings2.Metro),description_features_df],axis=1)
print X4.shape
X4.columns

(292, 1365)


Index([     u'Atlanta',       u'Austin',      u'Chicago',       u'Dallas',
             u'Denver',      u'Houston',  u'Los Angeles',        u'Miami',
           u'New York', u'Philadelphia',
       ...
              u'works',        u'world',       u'worlds',        u'write',
            u'writing',      u'written',         u'year',        u'years',
               u'york',           u'yr'],
      dtype='object', length=1365)

In [48]:
# Do logistic regression including all the title words identified by the count vectorizer
lr4 = LogisticRegression()      
model_lr4= lr4.fit(X4,y)

# Do cross validation and summarize the scores
lr4scores=cross_val_score(model_lr4,X4,y,cv=5)
print lr4scores
print "{} Score:\t{:0.3} ± {:0.3}".format("Logistic Regression", lr4scores.mean().round(3), lr4scores.std().round(3))

[ 0.81355932  0.79661017  0.72881356  0.70689655  0.68421053]
Logistic Regression Score:	0.746 ± 0.051


Accuracy is down and variance is up with this model vs one based just on the title.   
However we can use it to infer some words that might be impactful

In [49]:
# Put coefficients into readable format
coeff_vals4 = pd.DataFrame(lr4.coef_.transpose(),index = X4.columns,
                                   columns=['weights']).sort_values('weights',
                                                                       ascending=False)
# Raise e to the coefficient column to get the likelihood of being above the median
coeff_vals4['Likelihood_Change'] = np.exp(coeff_vals4['weights'])
coeff_vals4

,weights,Likelihood_Change
looking,1.077197,2.936437
San Francisco,0.970103,2.638216
analytical,0.929447,2.533107
senior,0.909150,2.482211
modeling,0.779938,2.181336
leading,0.778612,2.178446
join,0.767944,2.155330
model,0.698322,2.010377
Philadelphia,0.696852,2.007424
team,0.681965,1.977761


With Grid search rerun logistic regression with L1 regularization and range of C (inverse of alpha) to try to determine key variables in description

In [50]:
# Use features X4 from above (Metro/location + all words from the description)

In [51]:
#setup parameters
lr5 = LogisticRegression(penalty='l1')      
model_lr5= lr5.fit(X4,y)

Cs=np.array([0.001,0.01,0.1,1])

gridcv_lr5=GridSearchCV(estimator=lr5, param_grid=dict(C=Cs))
gridcv_lr5.fit(X4,y)

# grid.cv_results_   
gridcv_lr5.best_estimator_

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [52]:
# Best C was 1.0.  Will rerun with aditional values
Cs=np.array([0.5,0.8,1,2,10])

gridcv_lr5=GridSearchCV(estimator=lr5, param_grid=dict(C=Cs))
gridcv_lr5.fit(X4,y)

# grid.cv_results_   #not very interpretable output
gridcv_lr5.best_estimator_

LogisticRegression(C=10.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [53]:
# Best C was 10.  Will rerun with aditional values
Cs=np.array([8,10,15,20])

gridcv_lr5=GridSearchCV(estimator=lr5, param_grid=dict(C=Cs))
gridcv_lr5.fit(X4,y)

# grid.cv_results_   #not very interpretable output
gridcv_lr5.best_estimator_

LogisticRegression(C=8, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [54]:
#Best C was 8. Will rerun to hone in
Cs=np.array([5,6,7,8,9,10])

gridcv_lr5=GridSearchCV(estimator=lr5, param_grid=dict(C=Cs))
gridcv_lr5.fit(X4,y)

# grid.cv_results_   #not very interpretable output
gridcv_lr5.best_estimator_

LogisticRegression(C=5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [55]:
# Best C was 6.  Want to see associated coefficients into readable format
coeff_vals5 = pd.DataFrame(lr5.coef_.transpose(),index = X4.columns,
                                   columns=['weights']).sort_values('weights',
                                                                       ascending=False)
coeff_vals5

,weights
leading,1.886989
looking,1.840003
challenges,1.798919
San Francisco,1.736509
analytical,1.525006
modeling,1.450353
statistician,1.401144
model,1.400228
senior,1.301459
questions,1.281091


In [56]:
# Look at how many weights driven to 0
# coeff_vals5[coeff_vals5.weights==0]

In [57]:
# 1283 out 1365 weights driven to 0
# Look at weights that are not 0
# coeff_vals5[coeff_vals5.weights!=0]

In [58]:
# Write these to csv, manually remove those that I think are not meaningful 
# (like 'role') and then read back in file to work from going forward
coeff_vals5.to_csv('key features.csv')


These cities had coefficients driven to 0:
Seattle
Raleigh
Phoenix
New York
Los Angeles
Houston
Denver
Dallas
Interpreting that their salaries are near median

In [61]:
# Read edited file back in with 40 most impactful keywords from the description plus 8 impactful cities
keywords=pd.read_csv('key features2.csv',header=0)

In [62]:
# Rerun logistic regression to get new weights 
X6 = X4[keywords['Unnamed: 0'].tolist()]    #X6 is reduced list of keywords
lr6 = LogisticRegression()           
model_lr6= lr6.fit(X6,y)

# Do cross validation and summarize the scores
lr6scores=cross_val_score(model_lr6,X6,y,cv=5)
print lr6scores
print "{} Score:\t{:0.3} ± {:0.3}".format("Linear Regression", lr6scores.mean().round(3), lr6scores.std().round(3))

[ 0.77966102  0.77966102  0.77966102  0.77586207  0.73684211]
Linear Regression Score:	0.77 ± 0.017


In [63]:
# Similar to best scores I had previously
# Look at associated coefficients into readable format
coeff_vals6 = pd.DataFrame(lr6.coef_.transpose(),index = X6.columns,
                                   columns=['weights']).sort_values('weights',
                                                                       ascending=False)
# Raise e to the coefficient column to get the likelihood of being above the median
coeff_vals6['Likelihood_Change'] = np.exp(coeff_vals6['weights'])
coeff_vals6

,weights,Likelihood_Change
San Francisco,1.567672,4.795472
leading,1.326212,3.766747
analytical,1.314655,3.723466
innovative,1.279602,3.595208
Philadelphia,1.251906,3.497001
challenges,1.221201,3.391258
field,1.181571,3.259492
senior,1.178358,3.249035
model,1.173834,3.234370
modeling,1.120743,3.067132


In [64]:
# Write the words and weights to a csv file so key feed to Tableau
coeff_vals6.to_csv('final_keyword_values.csv')

##### Redo above without the cities - want key words from description only

In [66]:
# Edit previous file and read back in
keywordsD=pd.read_csv('key features3.csv',header=0)

In [67]:
# Rerun logistic regression to get new weights 
XkD = X4[keywordsD['Unnamed: 0'].tolist()]    #X6 is reduced list of keywords
lrkD = LogisticRegression()           
model_lrkD= lrkD.fit(XkD,y)

# Do cross validation and summarize the scores
lrkDscores=cross_val_score(model_lrkD,XkD,y,cv=5)
print lrkDscores
print "{} Score:\t{:0.3} ± {:0.3}".format("Linear Regression", lrkDscores.mean().round(3), lrkDscores.std().round(3))


[ 0.83050847  0.74576271  0.79661017  0.74137931  0.75438596]
Linear Regression Score:	0.774 ± 0.034


Similar accuracy and variance to other regressions and decision trees

In [68]:
# Look at associated coefficients into readable format
coeff_valskD = pd.DataFrame(lrkD.coef_.transpose(),index = XkD.columns,
                                   columns=['weights']).sort_values('weights',
                                                                       ascending=False)
# Raise e to the coefficient column to get the likelihood of being above the median
coeff_valskD['Likelihood_Change'] = np.exp(coeff_valskD['weights'])
coeff_valskD

,weights,Likelihood_Change
leading,1.497124,4.468818
innovative,1.420555,4.139417
model,1.291245,3.637311
analytical,1.214273,3.367845
client,1.174371,3.236106
modeling,1.172807,3.231050
challenges,1.155338,3.175097
senior,1.116352,3.053693
field,1.106593,3.024037
statistician,1.012160,2.751538


In [69]:
# Write the words and weights to a csv file so can feed to Tableau
coeff_valskD.to_csv('final_Description_values.csv')